In [ ]:
import pandas as pd
from pandas import DataFrame as df
import json
import pickle

In [ ]:
# load data from csv file, csv is extracted by running trained NS-VQA on CLEVR-min dataset
DATA_PATH = r'C:/Users/pegah/Desktop/programs_and_params_with_traces.csv'
data_raw = pd.read_csv(DATA_PATH)
data = data_raw.loc[data_raw.a_gt == data_raw.a_pr]

# modify data and get those needed
bad = ['relate[behind]','relate[front]','relate[left]','relate[right]','same_color','same_material',\
       'same_shape','same_size', 'union', 'intersect']
      
bad_idx = []
for i in range(len(data.p_trace)):
    try:
        for m in eval(data.p_trace[i]):
            try:
                a = m['token']
                if a in bad:
                    bad_idx.append(i)
                    break
            except:
                bad_idx.append(i)
                break
    except:
        bad_idx.append(i)
        

bad_idx.extend([149989,149990])  # somehow these wree left out!
copy_bad_idx = [i for i in bad_idx if i not in [15758, 36228]]
new_data = data.drop(copy_bad_idx)

# filter those whose answer only depends on previous step
bad_idx = []
for i in new_data.index:
    tok = [var['token'] for var in eval(new_data.p_trace[i]) if var['token'] != '<END>'][1:]
    if 'scene' in tok:
        bad_idx.append(i)
new_data = new_data.drop(bad_idx)

In [ ]:
# Config
cols = ['id', 'position', 'color', 'material', 'shape', 'size']
VQA_MODULES = ['count','equal_color','equal_integer','equal_material','equal_shape','equal_size','exist',\
'filter_color[blue]','filter_color[brown]','filter_color[cyan]','filter_color[gray]',\
'filter_color[green]','filter_color[purple]','filter_color[red]','filter_color[yellow]',\
'filter_material[rubber]','filter_material[metal]','filter_shape[cube]','filter_shape[cylinder]',\
'filter_shape[sphere]','filter_size[large]','filter_size[small]','greater_than','less_than',\
'query_color','query_material','query_shape','query_size','relate[behind]','relate[front]','relate[left]',\
'relate[right]','same_color','same_material','same_shape','same_size','unique']
COMPARE_MODULES = ['equal_color','equal_integer','equal_material','equal_shape','equal_size','greater_than','less_than']
FILTER_MODULES = ['filter_color[blue]','filter_color[brown]','filter_color[cyan]','filter_color[gray]',\
'filter_color[green]','filter_color[purple]','filter_color[red]','filter_color[yellow]',\
'filter_material[rubber]','filter_material[metal]','filter_shape[cube]','filter_shape[cylinder]',\
'filter_shape[sphere]','filter_size[large]','filter_size[small]']
QUERY_MODULES = ['query_color','query_material','query_shape','query_size']
RELATE_MODULES = ['relate[behind]','relate[front]','relate[left]','relate[right]']
SAME_MODULES = ['same_color','same_material','same_shape','same_size']
NPI_PROGRAMS = ['COUNT','COMPARE','MOVE_PTR','DELETE_ROW','QUERY','UNIQUE','EXIST','FILTER']
NPI_PROG_ID = {NPI_PROGRAMS[i]: i for i in range(len(NPI_PROGRAMS))}
ARG = {'COMPARE': {k:['EQ','GT', 'LT', 'NEQ'].index(k) for k in ['EQ','GT', 'LT', 'NEQ']},
        'FILTER': {1: {k.upper():cols[2:].index(k) for k in cols[2:]},
                   2: {k:['BLUE', 'BROWN', 'CYAN', 'GRAY', 'GREEN', 'PURPLE', 'RED', 'YELLOW', \
                         'RUBBER', 'METAL', 'CUBE', 'CYLINDER', 'SPHERE', 'LARGE', 'SMALL'].index(k) for\
                       k in ['BLUE', 'BROWN', 'CYAN', 'GRAY', 'GREEN', 'PURPLE', 'RED', 'YELLOW', \
                         'RUBBER', 'METAL', 'CUBE', 'CYLINDER', 'SPHERE', 'LARGE', 'SMALL']}},
        'QUERY': {k.upper():cols.index(k) for k in cols[1:]},
        'RELATE': {k:['BEHIND', 'FRONT', 'LEFT', 'RIGHT'].index(k) for k in ['BEHIND', 'FRONT', 'LEFT', 'RIGHT']},
        'SAME': {k.upper():cols[2:].index(k) for k in cols[2:]},
        'MOVE_PTR': {1: {k.upper():v for (k,v) in zip(cols, range(len(cols)))}, 2: {k:['DOWN','RESET'].index(k) for k in ['DOWN','RESET']}}
                  }
ARG_R = {'COMPARE': {k:v for (v,k) in ARG['COMPARE'].items()},\
        'FILTER': {1: {k:v for (v,k) in ARG['FILTER'][1].items()},\
                   2: {k:v for (v,k) in ARG['FILTER'][2].items()}},\
        'QUERY': {k:v for (v,k) in ARG['QUERY'].items()},\
        'RELATE': {k:v for (v,k) in ARG['RELATE'].items()},\
        'SAME': {k:v for (v,k) in ARG['SAME'].items()},\
        'MOVE_PTR': {1: {k:v for (v,k) in ARG['MOVE_PTR'][1].items()}, 2: {k:v for (v,k) in ARG['MOVE_PTR'][2].items()}}}
H_PROG = ['FILTER', 'RELATE', 'SAME', 'EXIST']


In [ ]:
# functions
def map_module(module):
    """ 
    Turn the output of VQA seq-to-seq parsing of the question (a string module) into NPI module. 
    return: (NPI_program, program_id) , [Arg]
    """
    if module not in VQA_MODULES:
        return 'error in map_module: unknown module'
    if module in COMPARE_MODULES: 
        tmp = ('COMPARE', NPI_PROG_ID['COMPARE'])
        arg1 = module.split('_')[0]
        if arg1 == 'equal':
            return tmp, [ARG['COMPARE']['EQ']]
        elif arg1 == 'greater':
            return tmp, [ARG['COMPARE']['GT']]
        elif arg1 == 'less':
            return tmp, [ARG['COMPARE']['LT']]
        else: 
            return 'error in map_modlue: unknown argument'
    elif module in FILTER_MODULES:
        [arg1 ,arg2] = module.split('_')[1].split('[') 
        arg2 = arg2[:-1]
        return ('FILTER', NPI_PROG_ID['FILTER']) , [ARG['FILTER'][1][arg1.upper()], ARG['FILTER'][2][arg2.upper()]]    
    elif module in QUERY_MODULES:
        arg1 = module.split('_')[1]
        return ('QUERY', NPI_PROG_ID['QUERY']) , [ARG['QUERY'][arg1.upper()]]
    elif module in RELATE_MODULES:
        arg1 = module.split('[')[1][:-1]
        return ('RELATE', NPI_PROG_ID['RELATE']), [ARG['RELATE'][arg1.upper()]]
    elif module in SAME_MODULES:
        arg1 = module.split('_')[1]
        return ('SAME', NPI_PROG_ID['SAME']), [ARG['SAME'][arg1.upper()]]
    else: 
        return (module.upper(), NPI_PROG_ID[module.upper()]) , []

# can improve by adding def of all lowr level functions used inside a class "scene", add "self" and call those here
def exec_prog(prog, args, scene, term):
    res = []
    if prog =='FILTER':
        if len(args) != 2:
            return ('error in executing FILTER: wrong number of args ({} instead of 2)'.format(len(args)))
        if len(scene) == 0:
            return ('error in executing FILTER: empty scene')
        arg1 = ARG_R[prog][1][args[0]]  # color,...
        arg2 = ARG_R[prog][2][args[1]]  # blue,...
#         # make new scene
#         mod = 'NEW_SCENE'
        new_s = []
#         res.append(((mod, NPI_PROG_ID[mod]),[],False))
        step = 0
        # loop over all objects (number of dicts in scene)
        for row in scene:
            step += 1
            # move col and row pointers one down
            mod =  'MOVE_PTR'
            res.append(((mod, NPI_PROG_ID[mod]),[ARG[mod][1][arg1], ARG[mod][2]['DOWN']],False))
            res.append(((mod, NPI_PROG_ID[mod]),[ARG[mod][1]['ID'], ARG[mod][2]['DOWN']],False))
            # compare
            mod = 'COMPARE'
            if row[arg1.lower()] != arg2.lower():
                res.append(((mod, NPI_PROG_ID[mod]),[ARG[mod]['EQ']],False))
                # add row to new_scene = new_s
                new_s.append(row)
                # check for terminattion
                mod = 'DELETE_ROW'                    
                res.append(((mod, NPI_PROG_ID[mod]),[],False))
            else:
                res.append(((mod, NPI_PROG_ID[mod]),[ARG[mod]['EQ']], False))
        mod =  'MOVE_PTR'
        res.append(((mod, NPI_PROG_ID[mod]),[ARG[mod][1][arg1], ARG[mod][2]['RESET']],False))
        res.append(((mod, NPI_PROG_ID[mod]),[ARG[mod][1]['ID'], ARG[mod][2]['RESET']],term))
#         mod = 'EXIT'
#         res.append(((mod, NPI_PROG_ID[mod]),[],term))
        return res, new_s
                           
    elif prog == 'EXIST':
        # count number of objects in a scene
        obj_num = len(scene)
        mod =  'COUNT'
        res.append(((mod, NPI_PROG_ID[mod]),[],False))
        # compare if number is greater than zero
        mod = 'COMPARE'
        res.append(((mod, NPI_PROG_ID[mod]),[ARG[mod]['GT']],term)) 
#         mod = 'EXIT'
#         res.append(((mod, NPI_PROG_ID[mod]),[],term))
        
        if obj_num > 0: 
            return res, 'yes'
        else:
            return res, 'no'
            
#     if prog =='RELATE'
#     if prog =='SAME'


def build_trace(module, s_prev, term):  
    """ 
        return [((,),[],False),..., ((,),[],True/False)] if higher level program, else just [((,),[],True/False)] 
    """
    tmp = map_module(module)  # (('FILTER', 8), [0, 1])
    prog = tmp[0][0]
    if prog in H_PROG: 
        result = [tmp + (False,)] 
        res, s_update = exec_prog(prog, tmp[1], s_prev, term)
        result.extend(res)
    else:
        result = [tmp + (term,)]
    return result

def build_npi_data(data):
    """ build training/testing data for inputs to npi (trace = execution trace)"""
    trace = []  # [(imgid_q1,trace_list), (imgid_q2, trace_list), ...]
    dbg = 0
    for (a_gt,imgid, vqa_trc, qid, qtype) in zip(data.a_gt, data.image_id, data.p_trace, data.q_id, data.q_type):
        terminate = False
        dbg += 1
        step = 0
        scene_prev = []
        trace_list = []  # for each row of p_trace(i.e. each question): [((,),[],False),..., ((,),[],True/False)]
        for mdl in eval(vqa_trc):
            step += 1
            if step == len(eval(vqa_trc)):
                terminate = True
            module = mdl['token']
            scene_now = mdl['ans']
            if module in ['<END>', 'scene',]:
                scene_prev = scene_now
                continue
            elif module in VQA_MODULES:
                try:
                    trace_list.extend(build_trace(module, scene_prev, terminate))
                    continue
                except:
                    print('error: undefined vqa module {} in step {}'.format(module, dbg))
                    break
            else: 
                print('error: unknown module in vqa trace')
                break
        trace.append((a_gt, imgid, qid, qtype, trace_list))
    return trace
                   

In [ ]:
# make train and test data (new.data.shape = (10044, 12))
train_data = new_data[:8000]
test_data = new_data[8000:]
# save train and test data
train_result = build_npi_data(train_data)
test_result = build_npi_data(test_data)
name = 'train'
with open(r'C:\Users\pegah\Desktop\vqa-npi\tasks\vqa\data\{}.pickle'.format(name), 'wb') as f:
    pickle.dump(train_result, f)
name = 'test'
with open(r'C:\Users\pegah\Desktop\vqa-npi\tasks\vqa\data\{}.pickle'.format(name), 'wb') as f:
    pickle.dump(test_result, f)

In [ ]:
# split data by the question type and question trace length 
# q by type
t_type = {}
for i in train_result:
    if i[3] not in t_type:
        t_type[i[3]]= [i]
    else:
        t_type[i[3]].append(i)
# q by len
t_len_less = []
t_len_more = []
for i in train_result:
    if len(i[4]) <= 20 :
        t_len_less.append(i)
    else:
        t_len_more.append(i)
        
train_query = t_type['query']
train_count = t_type['count']
train_exist = t_type['exist']
train_less20 = t_len_less
train_more20 = t_len_more

# save
name = 'train_query'
with open(r'C:\Users\pegah\Desktop\vqa-npi\tasks\vqa\data\{}.pickle'.format(name), 'wb') as f:
    pickle.dump(train_query, f)
name = 'train_count'
with open(r'C:\Users\pegah\Desktop\vqa-npi\tasks\vqa\data\{}.pickle'.format(name), 'wb') as f:
    pickle.dump(train_count, f)
name = 'train_exist'
with open(r'C:\Users\pegah\Desktop\vqa-npi\tasks\vqa\data\{}.pickle'.format(name), 'wb') as f:
    pickle.dump(train_exist, f)
name = 'train_less20'
with open(r'C:\Users\pegah\Desktop\vqa-npi\tasks\vqa\data\{}.pickle'.format(name), 'wb') as f:
    pickle.dump(train_less20, f)
name = 'train_more20'
with open(r'C:\Users\pegah\Desktop\vqa-npi\tasks\vqa\data\{}.pickle'.format(name), 'wb') as f:
    pickle.dump(train_more20, f)